In [30]:
import pandas as pd

# Load the dataset
file_path = 'final_changes.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head(), data.info(), data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10659 entries, 0 to 10658
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   10659 non-null  int64  
 1   gender                       10659 non-null  int64  
 2   age                          10659 non-null  int64  
 3   height(cm)                   10659 non-null  float64
 4   height(m)                    10659 non-null  float64
 5   weight(kg)                   10659 non-null  int64  
 6   activity_level               10659 non-null  float64
 7   calories_to_maintain_weight  10659 non-null  float64
 8   BMI                          10659 non-null  float64
 9   BMR                          10659 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 832.9 KB


(   Unnamed: 0  gender  age  height(cm)  height(m)  weight(kg)  activity_level  \
 0           1       1   28      172.72     1.7272          62             1.2   
 1           2       1   36      167.64     1.6764          59             1.7   
 2           4       1   34      165.10     1.6510          61             1.9   
 3           6       1   27      175.26     1.7526          65             1.9   
 4           8       0   45      172.72     1.7272          62             1.9   
 
    calories_to_maintain_weight        BMI       BMR  
 0                    1150.3008  20.782914  1564.500  
 1                    1585.0511  20.994073  1462.750  
 2                    1857.3982  22.378743  1476.875  
 3                    1794.9091  21.161563  1615.375  
 4                    1518.5351  20.782914  1479.500  ,
 None,
          Unnamed: 0        gender           age    height(cm)     height(m)  \
 count  10659.000000  10659.000000  10659.000000  10659.000000  10659.000000   
 mean   

In [31]:
# Remove the unnecessary 'Unnamed: 0' column
data_cleaned = data.drop(columns=['Unnamed: 0'])

# Split the dataset into 2 subsets for federated learning
subset1 = data_cleaned.sample(frac=0.5, random_state=42)  # First subset
subset2 = data_cleaned.drop(subset1.index)               # Second subset

# Show the size of each subset to ensure correct splitting
subset1_size, subset2_size = subset1.shape[0], subset2.shape[0]
subset1_size, subset2_size


(5330, 5329)

In [32]:
# Define features and target for BMI calculation (Dataset 1)
features_bmi = ['weight(kg)', 'height(m)']
target_bmi = 'BMI'

# Define features and target for BMR calculation (Dataset 2)
features_bmr = ['weight(kg)', 'height(cm)', 'age', 'gender']
target_bmr = 'BMR'

# Prepare the data for BMI prediction
X_bmi, y_bmi = subset1[features_bmi], subset1[target_bmi]

# Prepare the data for BMR prediction
X_bmr, y_bmr = subset2[features_bmr], subset2[target_bmr]

# Calculate BMR using the Mifflin-St Jeor equation
def calculate_bmr(row):
    if row['gender'] == 0:
        return 10 * row['weight(kg)'] + 6.25 * row['height(cm)'] - 5 * row['age'] + 5
    else:
        return 10 * row['weight(kg)'] + 6.25 * row['height(cm)'] - 5 * row['age'] - 161

# Apply the BMR calculation to the dataset
X_bmr['calculated_bmr'] = X_bmr.apply(calculate_bmr, axis=1)

# Display the first few rows of the datasets to confirm
X_bmi.head(), X_bmr.head()


C:\Users\rituk\AppData\Local\Temp\ipykernel_9428\3768796121.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_bmr['calculated_bmr'] = X_bmr.apply(calculate_bmr, axis=1)


(      weight(kg)  height(m)
 1953          68     1.7780
 1090          83     1.7018
 4139          50     1.5748
 1971          63     1.6764
 5422          70     1.7018,
    weight(kg)  height(cm)  age  gender  calculated_bmr
 1          59      167.64   36       1        1296.750
 2          61      165.10   34       1        1310.875
 4          62      172.72   45       0        1479.500
 5          50      160.02   27       1        1204.125
 6          53      160.02   65       0        1210.125)

In [33]:
from sklearn.linear_model import LinearRegression
import pandas as pd
# Create a linear regression model for BMI prediction
lr_bmi = LinearRegression()
lr_bmi.fit(X_bmi, y_bmi)

# Create a linear regression model for BMR prediction
lr_bmr = LinearRegression()
print(pd.DataFrame(data).drop(columns=['calories_to_maintain_weight']))

       Unnamed: 0  gender  age  height(cm)  height(m)  weight(kg)  \
0               1       1   28      172.72     1.7272          62   
1               2       1   36      167.64     1.6764          59   
2               4       1   34      165.10     1.6510          61   
3               6       1   27      175.26     1.7526          65   
4               8       0   45      172.72     1.7272          62   
...           ...     ...  ...         ...        ...         ...   
10654       23985       1   30      152.40     1.5240          52   
10655       23988       1   53      165.10     1.6510          47   
10656       23994       1   50      157.48     1.5748          49   
10657       23997       1   33      157.48     1.5748          49   
10658       23998       0   44      177.80     1.7780          61   

       activity_level        BMI       BMR  
0                 1.2  20.782914  1564.500  
1                 1.7  20.994073  1462.750  
2                 1.9  22.378743  14

In [34]:
from sklearn.model_selection import train_test_split


X_train1, X_test1, y_train1, y_test1 = train_test_split(X_bmi, y_bmi, test_size=0.2, random_state=0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_bmr, y_bmr, test_size=0.2, random_state=0)


In [35]:
from sklearn.discriminant_analysis import StandardScaler


scaler = StandardScaler()


In [36]:
print("Initial dataset summary:\n", data.describe(include='all'))
print("Missing values in the dataset:\n", data.isnull().sum())


Initial dataset summary:
          Unnamed: 0        gender           age    height(cm)     height(m)  \
count  10659.000000  10659.000000  10659.000000  10659.000000  10659.000000   
mean   11667.442349      0.519467     33.993808    165.625205      1.656252   
std     6815.550401      0.499644      8.030083      6.751584      0.067516   
min        1.000000      0.000000      0.000000    137.160000      1.371600   
25%     5958.500000      0.000000     29.000000    160.020000      1.600200   
50%    10711.000000      1.000000     32.000000    165.100000      1.651000   
75%    17457.000000      1.000000     37.000000    170.180000      1.701800   
max    23998.000000      1.000000    116.000000    190.500000      1.905000   

         weight(kg)  activity_level  calories_to_maintain_weight  \
count  10659.000000    10659.000000                 10659.000000   
mean      61.741721        1.487203                  2118.263117   
std        9.967163        0.239498                   388.

In [37]:
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)


In [38]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Input, Dense

def create_linear_regression_model1():
    model = Sequential([
        Input(shape=(X_train1.shape[1],)),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model


def create_linear_regression_model2():
    model = Sequential([
        Input(shape=(X_train2.shape[1],)),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model



In [39]:
import numpy as np


def federated_averaging(models, weights):
    avg_weights = [np.zeros_like(w) for w in models[0].get_weights()]
    total_weight = np.sum(weights)
    
    for model, weight in zip(models, weights):
        model_weights = model.get_weights()
        for i in range(len(avg_weights)):
            avg_weights[i] += model_weights[i] * weight
            
    for i in range(len(avg_weights)):
        avg_weights[i] /= total_weight

    return avg_weights

In [40]:
num_clients1 = 5
clients_data1 = np.array_split(X_train1, num_clients1)
clients_labels1 = np.array_split(y_train1, num_clients1)
client_weights1 = np.ones(num_clients1)  # Assuming equal we

num_clients2 = 5
clients_data2 = np.array_split(X_train2, num_clients2)
clients_labels2 = np.array_split(y_train2, num_clients2)
client_weights2 = np.ones(num_clients2)  # Assuming equal we

c:\Users\rituk\dev\venvs\FL\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [41]:
client_models1 = []
for client_data1, client_labels1 in zip(clients_data1, clients_labels1):
    model1 = create_linear_regression_model1()
    model1.fit(client_data1, client_labels1, epochs=5, batch_size=32, verbose=0)
    client_models1.append(model1)

client_models2 = []
for client_data2, client_labels2 in zip(clients_data2, clients_labels2):
    model2 = create_linear_regression_model2()
    model2.fit(client_data2, client_labels2, epochs=5, batch_size=32, verbose=0)
    client_models2.append(model2)




In [42]:
avg_weights1 = federated_averaging(client_models1, client_weights1)

avg_weights2 = federated_averaging(client_models2, client_weights2)

In [43]:
global_model1 = create_linear_regression_model1()
global_model1.set_weights(avg_weights1)

global_model2 = create_linear_regression_model2()
global_model2.set_weights(avg_weights2)

In [44]:
loss, accuracy = global_model1.evaluate(X_test1, y_test1, verbose=0)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

loss, accuracy = global_model2.evaluate(X_test2, y_test2, verbose=0)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

Test loss: 515.8008, Test accuracy: 22.4649
Test loss: 2219873.0000, Test accuracy: 1484.6556


In [45]:
global_model1.summary()

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

In [46]:
global_model2.summary()

Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6 (24.00 B)

 Trainable params: 6 (24.00 B)

 Non-trainable params: 0 (0.00 B)